In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('../data/train.csv')
df.head()


,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,4.0,2.0,4.0,0.0,5.0,1.0,0.0,3.0,2.0,2.0,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Dictionary

#### keep
- Demographics
- Physical measures
- PCIAT
- PreInt_EduHx-computerinternet_hoursday


In [2]:
def split_dataset(df, target_col, columns_to_keep):
    df = df.dropna(subset=target_col) # Drop rows with missing ssi
    df = df[columns_to_keep['categorical'] + columns_to_keep['numerical'] + [columns_to_keep['target']]]

    df_train_val = df.sample(frac=0.8, random_state=0)
    df_test = df.drop(df_train_val.index)
    df_train = df_train_val.sample(frac=0.8, random_state=1)
    df_validation = df_train_val.drop(df_train.index)
    
    return df_train, df_validation, df_test

columns_to_keep = {
    "categorical": [
        'Basic_Demos-Age',
        # 'Basic_Demos-Sex',
        # 'BIA-BIA_Activity_Level_num',
        'PreInt_EduHx-computerinternet_hoursday'
        ],
    "numerical": [
        'CGAS-CGAS_Score',
        'Physical-BMI',
    ],
    "target": "sii"
}

# df_train, df_validation, df_test = split_dataset(df, target_col=columns_to_keep['target'], columns_to_keep=columns_to_keep)


#### Missing values
#### Baseline with RF classifier

## Missing values

In [3]:
def fill_NAs(dataframe, columns, type='categorical'):
    if type == "categorical":
        return dataframe[columns].fillna("Missing").astype(str)
    return dataframe[columns].fillna(0)

# df_train[columns_to_keep['categorical']] = fill_NAs(df_train, columns_to_keep['categorical'], type="categorical")
# df_validation[columns_to_keep['categorical']] = fill_NAs(df_validation, columns_to_keep['categorical'], type="categorical")
# df_test[columns_to_keep['categorical']] = fill_NAs(df_test, columns_to_keep['categorical'], type="categorical")

# df_train[columns_to_keep['numerical']] = fill_NAs(df_train, columns_to_keep['numerical'], type="numerical")
# df_validation[columns_to_keep['numerical']] = fill_NAs(df_validation, columns_to_keep['numerical'], type="numerical")
# df_test[columns_to_keep['numerical']] = fill_NAs(df_test, columns_to_keep['numerical'], type="numerical")


## Baseline with RF classifier

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [4]:
ohe_pipe = make_pipeline(OneHotEncoder(handle_unknown='ignore'), StandardScaler())
ohe_pipe.fit(df_train[columns_to_keep['categorical']])

df_train_transformed = pd.DataFrame(
    ohe_pipe.transform(df_train[columns_to_keep['categorical']]).toarray(), 
    columns=ohe_pipe.get_feature_names_out()
)
# df_val_transformed = pd.DataFrame(
#     ohe_pipe.transform(df_validation[columns_to_keep['categorical']]).toarray(),
#     columns=ohe_pipe.get_feature_names_out()
# )
# df_test_transformed = pd.DataFrame(
#     ohe_pipe.transform(df_test[columns_to_keep['categorical']]).toarray(),
#     columns=ohe_pipe.get_feature_names_out()
# )

df_train_transformed[columns_to_keep['numerical']] = df_train[columns_to_keep['numerical']]
# df_val_transformed[columns_to_keep['numerical']] = df_validation[columns_to_keep['numerical']]
# df_test_transformed[columns_to_keep['numerical']] = df_test[columns_to_keep['numerical']]


In [5]:
# rf_model = RandomForestClassifier(n_estimators=100, random_state=0)
# rf_model.fit(df_train_transformed, df_train[columns_to_keep['target']])

# predictions = rf_model.predict(df_val_transformed)
# accuracy = accuracy_score(df_validation[columns_to_keep['target']], predictions)
# print(f'Accuracy: {accuracy:.4f}')

In [6]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# m = confusion_matrix(df_validation[target_col], predictions)
# ConfusionMatrixDisplay(m).plot()

## Age-model 
#### Two models > and < 12

In [69]:
def preprocess(df_train, df_validation, df_test, columns_to_keep):
# df_train_over, df_validation_over, df_test_under = split_dataset(df, target_col=columns_to_keep['target'], columns_to_keep=columns_to_keep)

    df_train[columns_to_keep['categorical']] = fill_NAs(df_train, columns_to_keep['categorical'], type="categorical")
    df_validation[columns_to_keep['categorical']] = fill_NAs(df_validation, columns_to_keep['categorical'], type="categorical")
    df_test[columns_to_keep['categorical']] = fill_NAs(df_test, columns_to_keep['categorical'], type="categorical")

    df_train[columns_to_keep['numerical']] = fill_NAs(df_train, columns_to_keep['numerical'], type="numerical")
    df_validation[columns_to_keep['numerical']] = fill_NAs(df_validation, columns_to_keep['numerical'], type="numerical")
    df_test[columns_to_keep['numerical']] = fill_NAs(df_test, columns_to_keep['numerical'], type="numerical")


    ohe_pipe = make_pipeline(OneHotEncoder(handle_unknown='ignore'))
    ohe_pipe.fit(df_train[columns_to_keep['categorical']])

    df_train_transformed = pd.DataFrame(
        ohe_pipe.transform(df_train[columns_to_keep['categorical']]).toarray(), 
        columns=ohe_pipe.get_feature_names_out()
    )
    df_val_transformed = pd.DataFrame(
        ohe_pipe.transform(df_validation[columns_to_keep['categorical']]).toarray(),
        columns=ohe_pipe.get_feature_names_out()
    )
    df_test_transformed = pd.DataFrame(
        ohe_pipe.transform(df_test[columns_to_keep['categorical']]).toarray(),
        columns=ohe_pipe.get_feature_names_out()
    )

    df_train_transformed[columns_to_keep['numerical']] = df_train[columns_to_keep['numerical']]
    df_val_transformed[columns_to_keep['numerical']] = df_validation[columns_to_keep['numerical']]
    df_test_transformed[columns_to_keep['numerical']] = df_test[columns_to_keep['numerical']]

    return df_train_transformed, df_val_transformed, df_test_transformed

def model(df_train_transformed, df_val_transformed, df_train, df_validation, instantiated_model):
    instantiated_model.fit(df_train_transformed, df_train[columns_to_keep['target']])
    predictions = instantiated_model.predict(df_val_transformed)
    accuracy = accuracy_score(df_validation[columns_to_keep['target']], predictions)
    # print(f'Accuracy: {accuracy:.4f}')
    return round(accuracy, 3)

    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

    m = confusion_matrix(df_validation[columns_to_keep['target']], predictions)
    ConfusionMatrixDisplay(m).plot()

In [92]:
df_under_8 = df[df['Basic_Demos-Age'] < 8]
df_over_8 = df[df['Basic_Demos-Age'] >= 8]
df_train, df_validation, df_test = split_dataset(df_over_8, target_col=columns_to_keep['target'], columns_to_keep=columns_to_keep)
df_train_transformed, df_val_transformed, df_test_transformed = preprocess(df_train, df_validation, df_test, columns_to_keep)

In [93]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent')
model(df_train_transformed, df_val_transformed, df_train, df_validation, dummy)


0.482

In [94]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=0)
model(df_train_transformed, df_val_transformed, df_train, df_validation, rf_model)

0.518

Dummy
- 0.482 - Over
- 0.781 - Under

Random Forest
- 0.518 - Over
- 0.781 - Under



In [63]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(df_train_transformed, df_train[columns_to_keep['target']])

predictions = dummy.predict(df_val_transformed)
accuracy = accuracy_score(df_validation[columns_to_keep['target']], predictions)
print(accuracy)

0.4819277108433735
